## Izvlačenje podataka za novu bazu i izračun novih varijabli iz postojećih

Izvlačenje klubova i njihovih id-eva, i zapisivanje u csv.

In [1]:
import csv
import pandas as pd
import os

fileDir = os.path.dirname(os.path.realpath('mozgalo_gladbach_eintracht_utakmice.csv'))
filename = os.path.join(fileDir, '../data/clean/mozgalo_gladbach_eintracht_utakmice.csv')
file = os.path.abspath(os.path.realpath(filename))

file1 = open(file, "r")

dict1 = {}
dict2 = {}

next(file1)
for line in file1:
    fields = line.split(",")
    dict1[int(fields[13])] = fields[1]
    dict2[int(fields[14])] = fields[2]

file1.close()

merged = {**dict1, **dict2}

file2 = open("klubovi.csv", "w", newline='')
writer = csv.writer(file2)
for key, value in merged.items():
    writer.writerow([key, value])

file2.close()

Priprema podataka za tablicu igraci_utakmice i zapis u csv.

In [3]:
import pandas as pd
import os

fileDir = os.path.dirname(os.path.realpath('mozgalo_gladbach_eintracht_akcije.csv'))
filename = os.path.join(fileDir, '../data/clean/mozgalo_gladbach_eintracht_akcije.csv')
file = os.path.abspath(os.path.realpath(filename))


dataframe = pd.read_csv(file, sep=",")

columns = dataframe[['gameID', 'playerID', 'playerPositionID']]
columnsunique = columns.drop_duplicates()
columnsunique.to_csv("igraci_utakmice.csv", sep=",", index=False)
columnsunique

,gameID,playerID,playerPositionID
0,6258,21913,17
1,6258,1992,18
2,6258,1892,15
3,6258,201366,14
4,6258,114354,13
6,6258,38433,16
8,6258,102537,21
21,6258,79430,4
39,6258,54482,1
45,6258,44079,15


Izračun novih varijabli, trajanje akcije, trajanje posjeda lopte za svakog igrača u akciji, sektor terena u kojem je počela akcija, sektor terena u kojem je završila akcija. Prvo su izračunati maksimum i minimum za koordinate akcija da znamo u kojim rasponima se kreću i da prema tome možemo kodirati sektore.

In [2]:
import pandas as pd
import os

fileDir = os.path.dirname(os.path.realpath('mozgalo_gladbach_eintracht_akcije.csv'))
filename = os.path.join(fileDir, '../Mozgalo/mozgalo_gladbach_eintracht_akcije.csv')
file = os.path.abspath(os.path.realpath(filename))


dataframe = pd.read_csv(file, sep=",")
dataframe['action_duration'] = dataframe['actionEndTime'] - dataframe['actionStartTime']
dataframe['poss_duration'] = dataframe['endPossTime'] - dataframe['startPossTime']



#ispis min i max vrijednosti da znam u kojim rasponima se krećem
maxx1 = dataframe['actionPosX1'].max()
maxx2 = dataframe['actionPosX2'].max()
maxy1 = dataframe['actionPosY1'].max()
maxy2 = dataframe['actionPosY2'].max()

minx1 = dataframe['actionPosX1'].min()
minx2 = dataframe['actionPosX2'].min()
miny1 = dataframe['actionPosY1'].min()
miny2 = dataframe['actionPosY2'].min()

print('actionPosX1 max: %s ' % maxx1)
print('actionPosX2 max: %s ' % maxx2)
print('actionPosY1 max: %s ' % maxy1)
print('actionPosY2 max: %s ' % maxy2)
print('\n')
print('actionPosX1 min: %s ' % minx1)
print('actionPosX2 min: %s ' % minx2)
print('actionPosY1 min: %s ' % miny1)
print('actionPosY2 min: %s ' % miny2)

#određivanje sektora početka akcije i spremanje u dataframe
for index, row in dataframe.iterrows():
    if((float(row['actionPosX1']) > 0) & (float(row['actionPosX1']) <= 350) &
      (float(row['actionPosY1']) > 0) & (float(row['actionPosY1']) <= 225)):
        sector = 1
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 350) & (float(row['actionPosX1']) <= 700) &
      (float(row['actionPosY1']) > 0) & (float(row['actionPosY1']) <= 225)):
        sector = 2
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 700) & (float(row['actionPosX1']) <= 1050) &
      (float(row['actionPosY1']) > 0) & (float(row['actionPosY1']) <= 225)):
        sector = 3
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 1050) & (float(row['actionPosX1']) <= 1400) &
      (float(row['actionPosY1']) > 0) & (float(row['actionPosY1']) <= 225)):
        sector = 4
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) > 0) & (float(row['actionPosX1']) <= 350) &
      (float(row['actionPosY1']) >= 225) & (float(row['actionPosY1']) <= 450)):
        sector = 5
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 350) & (float(row['actionPosX1']) <= 700) &
      (float(row['actionPosY1']) >= 225) & (float(row['actionPosY1']) <= 450)):
        sector = 6
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 700) & (float(row['actionPosX1']) <= 1050) &
      (float(row['actionPosY1']) >= 225) & (float(row['actionPosY1']) <= 450)):
        sector = 7
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 1050) & (float(row['actionPosX1']) <= 1400) &
      (float(row['actionPosY1']) >= 225) & (float(row['actionPosY1']) <= 450)):
        sector = 8
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) > 0) & (float(row['actionPosX1']) <= 350) &
      (float(row['actionPosY1']) >= 450) & (float(row['actionPosY1']) <= 675)):
        sector = 9
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 350) & (float(row['actionPosX1']) <= 700) &
      (float(row['actionPosY1']) >= 450) & (float(row['actionPosY1']) <= 675)):
        sector = 10
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 700) & (float(row['actionPosX1']) <= 1050) &
      (float(row['actionPosY1']) >= 450) & (float(row['actionPosY1']) <= 675)):
        sector = 11
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 1050) & (float(row['actionPosX1']) <= 1400) &
      (float(row['actionPosY1']) >= 450) & (float(row['actionPosY1']) <= 675)):
        sector = 12
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) > 0) & (float(row['actionPosX1']) <= 350) &
      (float(row['actionPosY1']) >= 675) & (float(row['actionPosY1']) <= 900)):
        sector = 13
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 350) & (float(row['actionPosX1']) <= 700) &
      (float(row['actionPosY1']) >= 675) & (float(row['actionPosY1']) <= 900)):
        sector = 14
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 700) & (float(row['actionPosX1']) <= 1050) &
      (float(row['actionPosY1']) >= 675) & (float(row['actionPosY1']) <= 900)):
        sector = 15
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) >= 1050) & (float(row['actionPosX1']) <= 1400) &
      (float(row['actionPosY1']) >= 675) & (float(row['actionPosY1']) <= 900)):
        sector = 16
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) == 0) | (float(row['actionPosY1']) == 0)):
        sector = 0
        dataframe.set_value(index,'start_sector',int(sector))
    elif((float(row['actionPosX1']) == 0) & (float(row['actionPosY1']) == 0)):
        sector = 0
        dataframe.set_value(index,'start_sector',int(sector))
        
#određivanje sektora završetka akcije i spremanje u dataframe

for index, row in dataframe.iterrows():
    if((float(row['actionPosX2']) > 0) & (float(row['actionPosX2']) <= 350) &
      (float(row['actionPosY2']) > 0) & (float(row['actionPosY2']) <= 225)):
        sector = 1
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 350) & (float(row['actionPosX2']) <= 700) &
      (float(row['actionPosY2']) > 0) & (float(row['actionPosY2']) <= 225)):
        sector = 2
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 700) & (float(row['actionPosX2']) <= 1050) &
      (float(row['actionPosY2']) > 0) & (float(row['actionPosY2']) <= 225)):
        sector = 3
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 1050) & (float(row['actionPosX2']) <= 1400) &
      (float(row['actionPosY2']) > 0) & (float(row['actionPosY2']) <= 225)):
        sector = 4
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) > 0) & (float(row['actionPosX2']) <= 350) &
      (float(row['actionPosY2']) >= 225) & (float(row['actionPosY2']) <= 450)):
        sector = 5
        dataframe.set_value(index,'end_sector',int(sector))    
    elif((float(row['actionPosX2']) >= 350) & (float(row['actionPosX2']) <= 700) &
      (float(row['actionPosY2']) >= 225) & (float(row['actionPosY2']) <= 450)):
        sector = 6
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 700) & (float(row['actionPosX2']) <= 1050) &
      (float(row['actionPosY2']) >= 225) & (float(row['actionPosY2']) <= 450)):
        sector = 7
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 1050) & (float(row['actionPosX2']) <= 1400) &
      (float(row['actionPosY2']) >= 225) & (float(row['actionPosY2']) <= 450)):
        sector = 8
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) > 0) & (float(row['actionPosX2']) <= 350) &
      (float(row['actionPosY2']) >= 450) & (float(row['actionPosY2']) <= 675)):
        sector = 9
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 350) & (float(row['actionPosX2']) <= 700) &
      (float(row['actionPosY2']) >= 450) & (float(row['actionPosY2']) <= 675)):
        sector = 10
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 700) & (float(row['actionPosX2']) <= 1050) &
      (float(row['actionPosY2']) >= 450) & (float(row['actionPosY2']) <= 675)):
        sector = 11
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 1050) & (float(row['actionPosX2']) <= 1400) &
      (float(row['actionPosY2']) >= 450) & (float(row['actionPosY2']) <= 675)):
        sector = 12
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) > 0) & (float(row['actionPosX2']) <= 350) &
      (float(row['actionPosY2']) >= 675) & (float(row['actionPosY2']) <= 900)):
        sector = 13
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 350) & (float(row['actionPosX2']) <= 700) &
      (float(row['actionPosY2']) >= 675) & (float(row['actionPosY2']) <= 900)):
        sector = 14
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 700) & (float(row['actionPosX2']) <= 1050) &
      (float(row['actionPosY2']) >= 675) & (float(row['actionPosY2']) <= 900)):
        sector = 15
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) >= 1050) & (float(row['actionPosX2']) <= 1400) &
      (float(row['actionPosY2']) >= 675) & (float(row['actionPosY2']) <= 900)):
        sector = 16
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) == 0) | (float(row['actionPosY2']) == 0)):
        sector = 0
        dataframe.set_value(index,'end_sector',int(sector))
    elif((float(row['actionPosX2']) == 0) & (float(row['actionPosY2']) == 0)):
        sector = 0
        dataframe.set_value(index,'end_sector',int(sector))

        
#određivanje sektora pozicije suca kada reagira na nešto u igri (pozicija suca)

for index, row in dataframe.iterrows():
    if((float(row['refPositionX']) > 0) & (float(row['refPositionX']) <= 350) &
      (float(row['refPositionY']) > 0) & (float(row['refPositionY']) <= 225)):
        sector = 1
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) > 350) & (float(row['refPositionX']) <= 700) &
      (float(row['refPositionY']) > 0) & (float(row['refPositionY']) <= 225)):
        sector = 2
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 700) & (float(row['refPositionX']) <= 1050) &
      (float(row['refPositionY']) > 0) & (float(row['refPositionY']) <= 225)):
        sector = 3
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 1050) & (float(row['refPositionX']) <= 1400) &
      (float(row['refPositionY']) > 0) & (float(row['refPositionY']) <= 225)):
        sector = 4
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) > 0) & (float(row['refPositionX']) <= 350) &
      (float(row['refPositionY']) >= 225) & (float(row['refPositionY']) <= 450)):
        sector = 5
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 350) & (float(row['refPositionX']) <= 700) &
      (float(row['refPositionY']) >= 225) & (float(row['refPositionY']) <= 450)):
        sector = 6
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 700) & (float(row['refPositionX']) <= 1050) &
      (float(row['refPositionY']) >= 225) & (float(row['refPositionY']) <= 450)):
        sector = 7
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 1050) & (float(row['refPositionX']) <= 1400) &
      (float(row['refPositionY']) >= 225) & (float(row['refPositionY']) <= 450)):
        sector = 8
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) > 0) & (float(row['refPositionX']) <= 350) &
      (float(row['refPositionY']) >= 450) & (float(row['refPositionY']) <= 675)):
        sector = 9
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 350) & (float(row['refPositionX']) <= 700) &
      (float(row['refPositionY']) >= 450) & (float(row['refPositionY']) <= 675)):
        sector = 10
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 700) & (float(row['refPositionX']) <= 1050) &
      (float(row['refPositionY']) >= 450) & (float(row['refPositionY']) <= 675)):
        sector = 11
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 1050) & (float(row['refPositionX']) <= 1400) &
      (float(row['refPositionY']) >= 450) & (float(row['refPositionY']) <= 675)):
        sector = 12
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) > 0) & (float(row['refPositionX']) <= 350) &
      (float(row['refPositionY']) >= 675) & (float(row['refPositionY']) <= 900)):
        sector = 13
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 350) & (float(row['refPositionX']) <= 700) &
      (float(row['refPositionY']) >= 675) & (float(row['refPositionY']) <= 900)):
        sector = 14
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 700) & (float(row['refPositionX']) <= 1050) &
      (float(row['refPositionY']) >= 675) & (float(row['refPositionY']) <= 900)):
        sector = 15
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) >= 1050) & (float(row['refPositionX']) <= 1400) &
      (float(row['refPositionY']) >= 675) & (float(row['refPositionY']) <= 900)):
        sector = 16
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) == 0) | (float(row['refPositionY']) == 0)):
        sector = 0
        dataframe.set_value(index,'ref_sector',int(sector))
    elif((float(row['refPositionX']) == 0) & (float(row['refPositionY']) == 0)):
        sector = 0
        dataframe.set_value(index,'ref_sector',int(sector))
        
dataframe.to_csv("mozgalo_gladbach_eintracht_akcije_novi_atributi.csv", sep=",", index=False)

actionPosX1 max: 1297 
actionPosX2 max: 1341 
actionPosY1 max: 836.32 
actionPosY2 max: 858.32 


actionPosX1 min: 0 
actionPosX2 min: 0 
actionPosY1 min: 0.0 
actionPosY2 min: 0.0 
